### Program to upload data into Milvus
End result is a Collection "demo_collection" with all the data vectorized 

In [1]:
# Program to upload data into Milvus
!pip install -U pymilvus
!pip install "pymilvus[model]"
!pip install --upgrade pymilvus transformers datasets torch
!pip install openpyxl
!pip install clean-text
# From
# https://milvus.io/docs/integrate_with_hugging-face.md

  Using cached pymilvus-2.4.5-py3-none-any.whl.metadata (5.6 kB)
  Using cached environs-9.5.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached milvus_lite-2.4.9-py3-none-manylinux2014_x86_64.whl.metadata (8.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.7 MB/s eta 0:00:00
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pymilvus-2.4.5-py3-none-any.whl (197 kB)
Using cached environs-9.5.0-py2.py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.8 MB/s eta 0:00:0000:0100:01
Using cached milvus_lite-2.4.9-py3-none-manylinux2014_x86_64.whl (49.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 67.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1

In [1]:
# Mivus
from pymilvus import MilvusClient, model,connections, db

# Huggingface
from datasets import Dataset, load_dataset # For loading
from transformers import AutoTokenizer, AutoModel

# Pytorch
import torch
import pandas as pd
from cleantext import clean

In [21]:
milHost="192.168.1.44"
milURI='http://192.168.1.44:19530'
milDBname="milvus_demo"
milCollection="demo_collection"
modelDimension= 384 # 768

In [83]:
df = pd.read_excel(io="/home/srini/AI-ML/CiscoTaskTracker.xlsx", sheet_name="Gemini-test",header=0)

In [84]:
# Drop columns, cleanup, fillNA, else tokanization fails
df.dropna(axis=0,how="all",inplace=True)
df.drop_duplicates(inplace=True)
#df = df.drop(df.columns[4], axis=1)
df = df.fillna("NA")
df.shape
df.info()
df.isna().sum()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 215 entries, 0 to 229
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Issue                 215 non-null    object
 1   Incident Description  215 non-null    object
 2   Responses             215 non-null    object
 3   Context               215 non-null    object
 4   Unnamed: 4            215 non-null    object
dtypes: object(5)
memory usage: 10.1+ KB


,Issue,Incident Description,Responses,Context,Unnamed: 4
0,User want to know is it possible to configure ...,"Hi Team,\nINC9078040\nissue: User want to know...",1)Opsmx replied as:-where in spinnaker it is a...,For dev pipelines user can add themselves in p...,NA
1,Spinnaker 4 Kubeconfig update 15 mins (Token e...,"Hi Opsmx ,\nMay i know if any testing is going...",Opsmx asked user to try once again.,monitor for 15 mins aftre token update the job...,NA
2,Error loading config file,"Hi Yeshashwini / OpsMx,\n\nDeployment are fail...",1)Opsmx suggested as need to check the Logs fo...,monitor for 15 mins aftre token update the job...,NA
3,\nDeployment failed due to error with Anypoint...,Hi Yeshashwini/ OpsMx\n\nINC9097548\nIssue: sp...,Opsmx suggested user need to check with anypo...,Need to check with anypoint platform team.,NA
4,"Exception in Dev env. related to GitHub ,\nRep...",Hi Yeshashwini/ Opsmx\nSpinnaker deployment is...,Opsmx suggested as:- \n1)Can you check with gi...,As the error clearly shows 404 status for gith...,NA


In [85]:
def cleanEveryCell(text) :
    return clean(text=text,
            fix_unicode=True,
            to_ascii=True,
            lower=True,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=False,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=False,
            no_punct=False,
            replace_with_punct="",
            replace_with_url="This is a URL",
            replace_with_email="Email",
            replace_with_phone_number="",
            replace_with_number="123",
            replace_with_digit="0",
            replace_with_currency_symbol="$",
            lang="en"
            )

df = df.applymap(cleanEveryCell)
df.head()

,Issue,Incident Description,Responses,Context,Unnamed: 4
0,user want to know is it possible to configure ...,"hi team, inc9078040 issue: user want to know i...",1)opsmx replied as:-where in spinnaker it is a...,for dev pipelines user can add themselves in p...,na
1,spinnaker 4 kubeconfig update 15 mins (token e...,"hi opsmx , may i know if any testing is going ...",opsmx asked user to try once again.,monitor for 15 mins aftre token update the job...,na
2,error loading config file,"hi yeshashwini / opsmx, deployment are failing...",1)opsmx suggested as need to check the logs fo...,monitor for 15 mins aftre token update the job...,na
3,deployment failed due to error with anypoint p...,hi yeshashwini/ opsmx inc9097548 issue: spinna...,opsmx suggested user need to check with anypoi...,need to check with anypoint platform team.,na
4,"exception in dev env. related to github , repe...",hi yeshashwini/ opsmx spinnaker deployment is ...,opsmx suggested as:- 1)can you check with gith...,as the error clearly shows 404 status for gith...,na


In [67]:
data = Dataset.from_pandas(df)

In [68]:
print(data)

Dataset({
    features: ['Issue', 'Incident Description', 'Responses', 'Context', 'Unnamed: 4', '__index_level_0__'],
    num_rows: 215
})


In [69]:
# This is model being used for RAG, not for inference
MODEL =  "sentence-transformers/all-MiniLM-L6-v2"  # Name of model from HuggingFace Models
INFERENCE_BATCH_SIZE = 64  # Batch size of model inference, may be used for uploading the docs into Milvus?

# Load tokenizer & model from HuggingFace Hub
ragTokenizer = AutoTokenizer.from_pretrained(MODEL) #This will be used to tokenize the inputs and uploading into Milvus
ragModel = AutoModel.from_pretrained(MODEL) # This should be used for finding relavent docs

/home/srini/AI-ML/srinitest/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [70]:
# Need to understand this, I am tokenizing only the incident description
def encode_text(batch):
    # Tokenize sentences
    encoded_input = ragTokenizer(
        batch["Incident Description"], padding=True, truncation=True, return_tensors="pt"
    )

    # Compute token embeddings
    with torch.no_grad():
        model_output = ragModel(**encoded_input)

    # Perform pooling
    token_embeddings = model_output[0]
    attention_mask = encoded_input["attention_mask"]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sentence_embeddings = torch.sum(
        token_embeddings * input_mask_expanded, 1
    ) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Normalize embeddings
    batch["description_embedding"] = torch.nn.functional.normalize(
        sentence_embeddings, p=2, dim=1
    )
    return batch


In [71]:
# Encode Data
data = data.map(encode_text, batched=True, batch_size=INFERENCE_BATCH_SIZE)
data_list = data.to_list()

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [72]:
# Create DB only once
#conn = connections.connect(host="192.168.1.44", port=19530)
#database = db.create_database("milvus_demo")

In [73]:
#client = MilvusClient(uri=milURI,db_name=milDBname, overwrite=True)

In [74]:
milvus_client = MilvusClient(milURI)
if milvus_client.has_collection(collection_name=milCollection):
    milvus_client.drop_collection(collection_name=milCollection)
milvus_client.create_collection(
    collection_name=milCollection,
    dimension=modelDimension,
    auto_id=True,  # Enable auto id
    enable_dynamic_field=True,  # Enable dynamic fields
    vector_field_name="description_embedding",  # Map vector field name and embedding column in dataset
    consistency_level="Strong",  # To enable search with latest data
)

In [75]:
#List Collections
#client.list_collections()
milvus_client.list_collections()

['demo_collection']

In [76]:
# Upload Data into Milvus
milvus_client.insert(collection_name=milCollection, data=data_list)

# Default embedding model
#embedding_fn = model.DefaultEmbeddingFunction()

{'insert_count': 215, 'ids': [451987299360597900, 451987299360597901, 451987299360597902, 451987299360597903, 451987299360597904, 451987299360597905, 451987299360597906, 451987299360597907, 451987299360597908, 451987299360597909, 451987299360597910, 451987299360597911, 451987299360597912, 451987299360597913, 451987299360597914, 451987299360597915, 451987299360597916, 451987299360597917, 451987299360597918, 451987299360597919, 451987299360597920, 451987299360597921, 451987299360597922, 451987299360597923, 451987299360597924, 451987299360597925, 451987299360597926, 451987299360597927, 451987299360597928, 451987299360597929, 451987299360597930, 451987299360597931, 451987299360597932, 451987299360597933, 451987299360597934, 451987299360597935, 451987299360597936, 451987299360597937, 451987299360597938, 451987299360597939, 451987299360597940, 451987299360597941, 451987299360597942, 451987299360597943, 451987299360597944, 451987299360597945, 451987299360597946, 451987299360597947, 4519872993

In [77]:
questions = {
    "Incident Description": [
        "deployments are failing",
    ]
}

# Generate question embeddings
question_embeddings = [v.tolist() for v in encode_text(questions)["description_embedding"]]

#print(question_embeddings)

In [78]:
# Now we search for our Incident description in Milvus
search_results = milvus_client.search(
    collection_name=milCollection,
    data=question_embeddings,
    limit=3,  # How many search results to output
    output_fields=["Responses", "Context"],  # Include these fields in search results
)


In [79]:
# Print out results
for q, res in zip(questions["Incident Description"], search_results):
    print("Incident Description:", q)
    for r in res:
        print(
            {
                "answer": r["entity"]["Responses"],
                "score": r["distance"],
            }
        )
    print("\n")

Incident Description: deployments are failing
{'answer': '1) cisco team has asked the user to re-try and deployment was successful', 'score': 0.7182451486587524}
{'answer': 'opsmx suggested as:- 1)cae-np-rtp2-919f5a this account is not present in clouddriver.yml can you check with user to add this account first and then re-execute the pipeline browse code tools: spinnaker tools / dynamic-clouddriver-accounts - gitscm (cisco.com)', 'score': 0.6687703728675842}
{'answer': 'opsmx suggested as because when pipeline ran(failed one) at time kube config was not update and it was not able to deploy every 15 min, as cae culster token being renewed', 'score': 0.6241561770439148}


